In [1]:
import torch
from sklearn.linear_model import LinearRegression
import numpy as np
import os
import string
import subprocess
import pandas as pd
import os

path = './programs'
if not os.path.exists(path):
    os.makedirs(path)


In [2]:
# import produce program
%run linear_regression.ipynb
%run boolean_regression.ipynb
%run symbolic_regression.ipynb

In [3]:
import pandas as pd

df = pd.DataFrame({'task': [],
                   'success': [],
                   'method': []})

df.to_csv('./results.csv')

In [4]:
tasks = [f for f in os.listdir('../tasks') if not f.startswith('.')]
tasks.sort()
tasks

['bit_addition',
 'rnn_abs_value_numerical',
 'rnn_abs_value_of_diff_numerical',
 'rnn_add_mod_3_numerical',
 'rnn_add_mod_4_numerical',
 'rnn_add_mod_5_numerical',
 'rnn_add_mod_6_numerical',
 'rnn_add_mod_7_numerical',
 'rnn_add_mod_8_numerical',
 'rnn_alternating_last3_numerical',
 'rnn_alternating_last4_numerical',
 'rnn_balanced_parenthesis_numerical',
 'rnn_base_3_addition',
 'rnn_base_4_addition',
 'rnn_base_5_addition',
 'rnn_base_6_addition',
 'rnn_base_7_addition',
 'rnn_base_8_addition',
 'rnn_bit_dot_prod_mod2_numerical',
 'rnn_bit_palindromes_numerical',
 'rnn_bit_shift_right_numerical',
 'rnn_bitwise_and',
 'rnn_bitwise_not',
 'rnn_bitwise_or',
 'rnn_bitwise_xor',
 'rnn_diff_last2_numerical',
 'rnn_diff_of_abs_value_numerical',
 'rnn_dithering_numerical',
 'rnn_div_3_numerical',
 'rnn_div_5_numerical',
 'rnn_div_7_numerical',
 'rnn_evens_counter_numerical',
 'rnn_evens_detector_numerical',
 'rnn_identity_numerical',
 'rnn_majority0_1_numerical',
 'rnn_majority0_2_numerica

In [5]:
import yaml

ii = 0

for task in tasks:
    
    
    df = pd.read_csv('./results.csv', index_col=0)

    ii += 1
    
    method = "None"
    print('-------------------------------------------------')
    print(task)
    with open(f'../tasks/{task}/args.yaml', 'r') as file:
        args = yaml.safe_load(file)
        
    hidden_dim = args['hidden_dim']
    print("hidden_dim = ", hidden_dim)
    rounding_patterns = list(product(*[['+','-']]*hidden_dim))
    
    if hidden_dim > 10:
        print("hidden_dim too large, exit...")
        df = pd.read_csv('./results.csv', index_col=0)
        item = {'task': task, 'success': 0, 'method': method} 
        df = df.append(item, ignore_index = True)
        df.to_csv('./results.csv')
        continue
    
    # create dataset
    print('1/5: create dataset')
    subprocess.run(["python", f"../tasks/{task}/create_dataset.py"])
    
    
    #ckpts = ['tasks', 'models_regularized', 'models_hammered']
    ckpts = ['models_hammered']
    
    for ckpt in ckpts:
        
        exist = os.path.isfile('../models_regularized' + '/' + task + '/regularized_model_best.pt')
        if exist == False and ckpt == 'models_regularized':
            print('not exist')
            continue
            
        print('ckpt = ', ckpt)

        print('2/5: lattice')
        # compute lattice
        subprocess.run(["python", "auto_encode_RNN.py", "--task", task, '--ckpt', ckpt])

        print('3/5: Linear regression')
        code, hidden_dim = produce_program_linear_regression(task, print_code=False);
        exec(code)
        if success_flag == 1:
            method = "LR"

        if success_flag == 0:

            print('4/5: Boolean regression')
            code = produce_program_boolean_regression(task, print_code=False, ckpt=ckpt); # also try eff_dim_increase = 1
            exec(code)
            success_flag
            if success_flag == 1:
                method = "BR"

            else:
                code = produce_program_boolean_regression(task, effective_hidden_dim_increase=1, print_code=False); # also try eff_dim_increase = 1
                exec(code)
                print(success_flag)

                if success_flag == 1:
                    method = "BR"


                else:
                    print('5/5: Symbolic regression')
                    code, function_strings = produce_program_symbolic_regression(task, print_code=False);
                    exec(code)
                    success_flag
                    if success_flag == 1:
                        method = "SR"
                    else:
                        if hidden_dim < 3:
                            for h_round in rounding_patterns:
                                print('round pattern:', h_round)
                                code, function_strings = produce_program_symbolic_regression(task, print_code=False, h_round=h_round, function_strings=function_strings);
                                exec(code)
                                success_flag
                                if success_flag == 1:
                                    method = "SR"
                                    break
                                    
        if success_flag == 1:
            break

    df = pd.read_csv('./results.csv', index_col=0)
    item = {'task': task, 'success': success_flag, 'method': method} 
    df = df.append(item, ignore_index = True)
    df.to_csv('./results.csv')




-------------------------------------------------
bit_addition
hidden_dim =  2
1/5: create dataset
ckpt =  models_hammered
2/5: lattice


/Users/ziming/Desktop/2022/research/code/neural-verification-regression-github/regression/integer_autoencoder.py:18: RuntimeWarning: invalid value encountered in divide
  proj = np.round(np.sum(basis*basis[[i]],axis=1)/np.linalg.norm(basis[i])**2)


[0.001 0.001]
saving txt
3/5: Linear regression
hidden_dim=2
input_dim > 1, skip...
4/5: Boolean regression
verification success
-------------------------------------------------
rnn_abs_value_numerical
hidden_dim =  1
1/5: create dataset
ckpt =  models_hammered
2/5: lattice


/var/folders/6j/b6y80djd4nb5hl73rv3sv8y80000gn/T/ipykernel_99359/3324284619.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(item, ignore_index = True)


[0.1   0.001]
saving txt
3/5: Linear regression
hidden_dim=1
verification failure
4/5: Boolean regression
dim too high (perm > 16!), skip...
dim too high (perm > 16!), skip...
0
5/5: Symbolic regression
trying parity  ('+',)


./symbolic_regression/main.cpp:32:1: warning: non-void function does not return a value in all control paths [-Wreturn-type]
};
^
1 warning generated.


symbolic regressing


KeyboardInterrupt: 